# 📌 **01 - Pinecone: Base Vetorial**

## 🎯 **Objetivo:**
Configurar e popular o Pinecone com dados turísticos para Rio e Paris.

## 📋 **O que faremos:**
1. ⚙️ Setup do Pinecone
2. 📊 Criação do índice vetorial  
3. 📝 Indexação dos dados turísticos
4. ✅ Testes de conectividade

---

## 1️⃣ **Setup e Configuração**

In [1]:
# Imports necessários
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
import time

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 2️⃣ **Configuração do Pinecone**

In [2]:
# Configuração das credenciais
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT', 'us-east-1')
INDEX_NAME = "turismo-inteligente"

if not PINECONE_API_KEY:
    print("⚠️ Configure PINECONE_API_KEY no arquivo .env")
else:
    print("✅ Credenciais carregadas!")

✅ Credenciais carregadas!


In [3]:
# Inicializar cliente Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Verificar índices existentes
print("📋 Índices existentes:")
print()
print("=" * 20)

for index in pc.list_indexes().names():
    print(f" [] {index}")
    print("=" * 20)

📋 Índices existentes:

 [] guia-viagem
 [] ia-na-pratica
 [] turismo-inteligente


In [4]:
# Criar índice se não existir
if INDEX_NAME not in pc.list_indexes().names():
    print(f"🔨 Criando índice '{INDEX_NAME}'...")
    
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # Para sentence-transformers/all-MiniLM-L6-v2
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region=PINECONE_ENVIRONMENT
        )
    )
    
    # Aguardar inicialização
    while not pc.describe_index(INDEX_NAME).status['ready']:
        time.sleep(1)
        print("⏳ Aguardando inicialização...")
    
    print("✅ Índice criado com sucesso!")
else:
    print(f"✅ Índice '{INDEX_NAME}' já existe!")

# Conectar ao índice
index = pc.Index(INDEX_NAME)
print(f"🔗 Conectado ao índice: {INDEX_NAME}")

✅ Índice 'turismo-inteligente' já existe!
🔗 Conectado ao índice: turismo-inteligente


## 🗑️ **Limpeza de Índices (Opcional)**

⚠️ **ATENÇÃO:** Código de segurança para remoção de índices. Use apenas quando necessário!

In [5]:
# 🚨 CÓDIGO DE LIMPEZA - DESCOMENTE APENAS QUANDO NECESSÁRIO 🚨

# Opção 1: Apagar índice específico por nome
# ⚠️ CONFIGURE O NOME DO ÍNDICE AQUI ANTES DE DESCOMENTAR:
index_para_apagar = "rag-musculacao"  # ← MODIFIQUE ESTE NOME CONFORME NECESSÁRIO

# if index_para_apagar in pc.list_indexes().names():
#     print(f"🗑️ Apagando índice '{index_para_apagar}'...")
#     pc.delete_index(index_para_apagar)
#     print(f"✅ Índice '{index_para_apagar}' removido com sucesso!")
# else:
#     print(f"⚠️ Índice '{index_para_apagar}' não existe.")
#     print("📋 Índices disponíveis:", list(pc.list_indexes().names()))




# Opção 2: Apagar TODOS os índices da conta (MUITO PERIGOSO!)
# CUIDADO: Isso apagará TODOS os seus índices Pinecone!

# print("🚨 ATENÇÃO: Apagando TODOS os índices...")
# indices = pc.list_indexes().names()

# if not indices:
#     print("✅ Nenhum índice encontrado.")
# else:
#     for index_name in indices:
#         print(f"🗑️ Apagando: {index_name}")
#         pc.delete_index(index_name)
    
#     print(f"💀 TODOS os {len(indices)} índices foram removidos!")


print("🔒 Código de limpeza disponível (comentado por segurança)")
print("📝 ANTES DE USAR: Modifique os nomes dos índices nas variáveis acima")
print("🎯 Depois descomente apenas a opção que precisar")

🔒 Código de limpeza disponível (comentado por segurança)
📝 ANTES DE USAR: Modifique os nomes dos índices nas variáveis acima
🎯 Depois descomente apenas a opção que precisar


## 3️⃣ **Dados Turísticos**

In [6]:
def carregar_base_conhecimento(arquivo_path: str = "base_conhecimento.txt") -> dict:
    """
    Carrega a base de conhecimento do arquivo txt e organiza em dicionário.
    
    Returns:
        Dicionário com dados organizados por categoria
    """
    try:
        with open(arquivo_path, 'r', encoding='utf-8') as file:
            conteudo = file.read()
        
        dados = {
            "rio_roteiros": [],
            "rio_logistica": [],
            "paris_roteiros": [],
            "paris_logistica": []
        }
        
        linhas = conteudo.split('\n')
        secao_atual = None
        
        for linha in linhas:
            linha = linha.strip()
            if not linha or linha.startswith('#'):
                continue
                
            # Identificar seções (com ou sem ##)
            if 'ROTEIROS - RIO' in linha:
                secao_atual = "rio_roteiros"
                continue
            elif 'LOGÍSTICA - RIO' in linha:
                secao_atual = "rio_logistica"
                continue
            elif 'ROTEIROS - PARIS' in linha:
                secao_atual = "paris_roteiros"
                continue
            elif 'LOGÍSTICA - PARIS' in linha:
                secao_atual = "paris_logistica"
                continue
            elif secao_atual and ':' in linha and not linha.startswith('#'):
                # Adicionar linha de dados à seção atual (sem chunking aqui)
                dados[secao_atual].append(linha)
        
        return dados
        
    except FileNotFoundError:
        print("⚠️ Arquivo base_conhecimento.txt não encontrado. Usando dados padrão.")
        # Fallback com dados básicos
        return {
            "rio_roteiros": [
                "Cristo Redentor: Uma das Sete Maravilhas do Mundo Moderno.",
                "Pão de Açúcar: Bondinho famoso com vista panorâmica."
            ],
            "rio_logistica": [
                "Aeroporto Galeão: Principal aeroporto internacional.",
                "Metro Rio: Conecta principais pontos turísticos."
            ],
            "paris_roteiros": [
                "Torre Eiffel: Símbolo de Paris, 324m de altura.",
                "Museu do Louvre: Maior museu do mundo."
            ],
            "paris_logistica": [
                "Charles de Gaulle: Principal aeroporto.",
                "Metro Paris: 14 linhas disponíveis."
            ]
        }

# Carregar base de conhecimento do arquivo externo
dados_turismo = carregar_base_conhecimento()

print("📚 **BASE DE CONHECIMENTO CARREGADA:**")
for categoria, docs in dados_turismo.items():
    cidade = "RIO" if "rio" in categoria else "PARIS"
    tipo = "ROTEIROS" if "roteiros" in categoria else "LOGÍSTICA"
    print(f"   🏷️ {cidade} - {tipo}: {len(docs)} documentos")

total_docs = sum(len(docs) for docs in dados_turismo.values())
print(f"\n📊 Total de documentos: {total_docs}")
print("✅ **Dados carregados de arquivo externo: base_conhecimento.txt**")

📚 **BASE DE CONHECIMENTO CARREGADA:**
   🏷️ RIO - ROTEIROS: 5 documentos
   🏷️ RIO - LOGÍSTICA: 6 documentos
   🏷️ PARIS - ROTEIROS: 5 documentos
   🏷️ PARIS - LOGÍSTICA: 5 documentos

📊 Total de documentos: 21
✅ **Dados carregados de arquivo externo: base_conhecimento.txt**


## 4️⃣ **Chunking dos Dados**

⚙️ **Processamento:** Dividir documentos grandes em chunks menores para melhor performance dos embeddings.

In [7]:
def aplicar_chunking(texto: str, max_chars: int = 200) -> list:
    """
    Aplica estratégia de chunking em um texto.
    
    Args:
        texto: Texto para fazer chunking
        max_chars: Máximo de caracteres por chunk
        
    Returns:
        Lista de chunks
    """
    # Se o texto é pequeno, retorna como está
    if len(texto) <= max_chars:
        return [texto]
    
    chunks = []
    
    # Estratégia 1: Dividir por sentenças (ponto final)
    sentences = texto.split('. ')
    current_chunk = ""
    
    for sentence in sentences:
        # Adicionar ponto final de volta (exceto na última)
        if sentence != sentences[-1]:
            sentence += "."
            
        # Se adicionar esta sentença ultrapassar o limite
        if len(current_chunk + sentence) > max_chars and current_chunk:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += " " + sentence if current_chunk else sentence
    
    # Adicionar último chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    # Se ainda houver chunks muito grandes, dividir por vírgula
    final_chunks = []
    for chunk in chunks:
        if len(chunk) > max_chars:
            # Dividir por vírgula
            parts = chunk.split(', ')
            temp_chunk = ""
            
            for part in parts:
                if len(temp_chunk + part) > max_chars and temp_chunk:
                    final_chunks.append(temp_chunk.strip())
                    temp_chunk = part
                else:
                    temp_chunk += ", " + part if temp_chunk else part
                    
            if temp_chunk:
                final_chunks.append(temp_chunk.strip())
        else:
            final_chunks.append(chunk)
    
    return final_chunks if final_chunks else [texto]

# Aplicar chunking nos dados carregados
print("🔄 **APLICANDO CHUNKING NOS DADOS...**\n")

dados_com_chunks = {}
total_chunks = 0

for categoria, documentos in dados_turismo.items():
    cidade = "RIO" if "rio" in categoria else "PARIS"
    tipo = "ROTEIROS" if "roteiros" in categoria else "LOGÍSTICA"
    
    print(f"📝 **{cidade} - {tipo}:**")
    
    chunks_categoria = []
    
    for i, doc in enumerate(documentos):
        print(f"   📄 Doc {i+1}: {doc[:50]}...")
        
        # Aplicar chunking
        chunks = aplicar_chunking(doc, max_chars=200)
        
        print(f"      ➜ {len(chunks)} chunk(s) gerado(s)")
        for j, chunk in enumerate(chunks):
            print(f"        {j+1}. [{len(chunk)} chars] {chunk}")
            
        chunks_categoria.extend(chunks)
        total_chunks += len(chunks)
    
    dados_com_chunks[categoria] = chunks_categoria
    print()

print(f"📊 **RESUMO DO CHUNKING:**")
print(f"   📈 Total de chunks: {total_chunks}")
print(f"   📏 Limite por chunk: 200 caracteres")
print(f"   🎯 Estratégia: sentenças → vírgulas → força bruta")
print("✅ **Chunking concluído com sucesso!**")

🔄 **APLICANDO CHUNKING NOS DADOS...**

📝 **RIO - ROTEIROS:**
   📄 Doc 1: Cristo Redentor: Uma das Sete Maravilhas do Mundo ...
      ➜ 1 chunk(s) gerado(s)
        1. [83 chars] Cristo Redentor: Uma das Sete Maravilhas do Mundo Moderno, localizado no Corcovado.
   📄 Doc 2: Pão de Açúcar: Bondinho famoso com vista panorâmic...
      ➜ 1 chunk(s) gerado(s)
        1. [71 chars] Pão de Açúcar: Bondinho famoso com vista panorâmica da cidade e praias.
   📄 Doc 3: Praia de Copacabana: 4km de areia branca, calçadão...
      ➜ 1 chunk(s) gerado(s)
        1. [75 chars] Praia de Copacabana: 4km de areia branca, calçadão famoso, hotéis luxuosos.
   📄 Doc 4: Praia de Ipanema: Bairro sofisticado, praia das ce...
      ➜ 1 chunk(s) gerado(s)
        1. [86 chars] Praia de Ipanema: Bairro sofisticado, praia das celebridades, pôr do sol inesquecível.
   📄 Doc 5: Santa Teresa: Bairro boêmio, casarões coloniais, a...
      ➜ 1 chunk(s) gerado(s)
        1. [69 chars] Santa Teresa: Bairro boêmio, casarõ

## 5️⃣ **Setup dos Embeddings**

In [8]:
# Inicializar modelo de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("🧠 Modelo de embeddings carregado!")
print(f"📏 Dimensão: {len(embeddings.embed_query('teste'))}")

🧠 Modelo de embeddings carregado!
📏 Dimensão: 384


## 6️⃣ **Indexação Automática no Pinecone**

🔄 **Sistema Simplificado:** Sempre limpa e reindexiza para garantir dados atualizados.

In [9]:
# 🔄 SISTEMA SIMPLES: Limpar e Reindexar SEMPRE
print("🧹 Limpando dados existentes...")

# Apagar todos os vetores do índice
index.delete(delete_all=True)
print("✅ Dados anteriores removidos!")

# Aguardar limpeza
time.sleep(2)

print("🚀 Iniciando nova indexação...")

vectors_to_upsert = []

for categoria, documentos in dados_com_chunks.items():
    for i, doc in enumerate(documentos):
        # Gerar embedding
        embedding = embeddings.embed_query(doc)
        
        # Criar ID único
        vector_id = f"{categoria}_{i}"
        
        # Preparar vetor com metadados
        vector_data = {
            "id": vector_id,
            "values": embedding,
            "metadata": {
                "text": doc,
                "categoria": categoria,
                "cidade": "rio" if "rio" in categoria else "paris",
                "tipo": "roteiros" if "roteiros" in categoria else "logistica"
            }
        }
        
        vectors_to_upsert.append(vector_data)
        print(f"📝 Preparado: {vector_id}")

# Fazer upsert em lote
print(f"📤 Inserindo {len(vectors_to_upsert)} vetores...")
index.upsert(vectors=vectors_to_upsert)

print("✅ Indexação completa!")
print("🎯 Sistema sempre atualizado com dados mais recentes!")

🧹 Limpando dados existentes...
✅ Dados anteriores removidos!
🚀 Iniciando nova indexação...
📝 Preparado: rio_roteiros_0
📝 Preparado: rio_roteiros_1
📝 Preparado: rio_roteiros_2
📝 Preparado: rio_roteiros_3
📝 Preparado: rio_roteiros_4
📝 Preparado: rio_logistica_0
📝 Preparado: rio_logistica_1
📝 Preparado: rio_logistica_2
📝 Preparado: rio_logistica_3
📝 Preparado: rio_logistica_4
📝 Preparado: rio_logistica_5
📝 Preparado: paris_roteiros_0
📝 Preparado: paris_roteiros_1
📝 Preparado: paris_roteiros_2
📝 Preparado: paris_roteiros_3
📝 Preparado: paris_roteiros_4
📝 Preparado: paris_logistica_0
📝 Preparado: paris_logistica_1
📝 Preparado: paris_logistica_2
📝 Preparado: paris_logistica_3
📝 Preparado: paris_logistica_4
📤 Inserindo 21 vetores...
✅ Indexação completa!
🎯 Sistema sempre atualizado com dados mais recentes!


## 7️⃣ **Teste de Conectividade**

In [10]:
# Aguardar propagação dos dados
time.sleep(2)

# Verificar estatísticas finais
stats = index.describe_index_stats()
print("📊 **ESTATÍSTICAS DO ÍNDICE:**")
print(f"   📈 Total de vetores: {stats['total_vector_count']}")
print(f"   🏷️ Namespaces: {list(stats.get('namespaces', {}).keys()) if stats.get('namespaces') else 'default'}")
print(f"   💾 Dimensão: {stats.get('dimension', 'N/A')}")

📊 **ESTATÍSTICAS DO ÍNDICE:**
   📈 Total de vetores: 0
   🏷️ Namespaces: default
   💾 Dimensão: 384


In [11]:
# Teste de busca simples
query = "pontos turísticos famosos"
query_embedding = embeddings.embed_query(query)

# Buscar documentos similares
results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

print(f"🔍 **TESTE DE BUSCA:** '{query}'")
print("\n📋 **RESULTADOS:**")
for i, match in enumerate(results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    cidade = match['metadata']['cidade']
    
    print(f"\n{i}. **{cidade.upper()}** (Score: {score:.3f})")
    print(f"   {text}")

🔍 **TESTE DE BUSCA:** 'pontos turísticos famosos'

📋 **RESULTADOS:**


## ✅ **Resumo do Notebook 01:**

### 🎯 **O que fizemos:**
- ⚙️ Configuramos o Pinecone Cloud
- 🗄️ Criamos índice vetorial "turismo-inteligente"
- 📚 Indexamos 20 documentos turísticos (Rio + Paris)
- 🧠 Usamos embeddings sentence-transformers
- ✅ Testamos busca por similaridade

### 📊 **Dados Indexados:**
- 🇧🇷 **Rio**: 10 docs (5 roteiros + 5 logística)
- 🇫🇷 **Paris**: 10 docs (5 roteiros + 5 logística)
- 📏 **Dimensão**: 384 (all-MiniLM-L6-v2)

### ➡️ **Próximo Passo:**
**02-RAG.ipynb** → Sistema de recuperação inteligente

---
✨ **Base vetorial pronta para uso!**

## 8️⃣ **Visualização dos Dados Carregados**

In [12]:
# Demonstrar o conteúdo carregado do arquivo
print("📋 **CONTEÚDO DETALHADO DA BASE DE CONHECIMENTO:**\\n")

for categoria, documentos in dados_turismo.items():
    cidade = "🏖️ RIO DE JANEIRO" if "rio" in categoria else "🗼 PARIS"
    tipo = "ROTEIROS" if "roteiros" in categoria else "LOGÍSTICA"
    
    print(f"{cidade} - {tipo}:")
    for i, doc in enumerate(documentos, 1):
        print(f"   {i}. {doc}")
    print()

# Calcular estatísticas básicas
total_geral = sum(len(docs) for docs in dados_turismo.values())
rio_total = sum(len(docs) for cat, docs in dados_turismo.items() if "rio" in cat)
paris_total = sum(len(docs) for cat, docs in dados_turismo.items() if "paris" in cat)
roteiros_total = sum(len(docs) for cat, docs in dados_turismo.items() if "roteiros" in cat)
logistica_total = sum(len(docs) for cat, docs in dados_turismo.items() if "logistica" in cat)

print("📊 **ESTATÍSTICAS DETALHADAS:**")
print(f"   📈 Total geral: {total_geral} documentos")
print(f"   🏙️ Por cidade: Rio ({rio_total}) | Paris ({paris_total})")
print(f"   📝 Por tipo: Roteiros ({roteiros_total}) | Logística ({logistica_total})")

print("\\n🎯 **Vantagem:** Dados centralizados e fáceis de manter!")

📋 **CONTEÚDO DETALHADO DA BASE DE CONHECIMENTO:**\n
🏖️ RIO DE JANEIRO - ROTEIROS:
   1. Cristo Redentor: Uma das Sete Maravilhas do Mundo Moderno, localizado no Corcovado.
   2. Pão de Açúcar: Bondinho famoso com vista panorâmica da cidade e praias.
   3. Praia de Copacabana: 4km de areia branca, calçadão famoso, hotéis luxuosos.
   4. Praia de Ipanema: Bairro sofisticado, praia das celebridades, pôr do sol inesquecível.
   5. Santa Teresa: Bairro boêmio, casarões coloniais, ateliês de artistas.

🏖️ RIO DE JANEIRO - LOGÍSTICA:
   1. Aeroporto Galeão: Principal aeroporto internacional, 20km do centro.
   2. Aeroporto Santos Dumont: Aeroporto doméstico no centro da cidade, voos nacionais, vista privilegiada da Baía de Guanabara.
   3. Metro Rio: Linhas 1, 2 e 4 conectam principais pontos turísticos.
   4. Uber/99: Disponível 24h, mais seguro que táxi comum.
   5. BRT: Ônibus rápido conecta Barra da Tijuca ao centro.
   6. Hospedagem Copacabana: Hotéis de todas as categorias na orla.

🗼 P